In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:



df = pd.read_csv('injury-details-2025-q3.csv')


london_boroughs = [
    'Barking & Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
    'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich',
    'Hackney', 'Hammersmith & Fulham', 'Haringey', 'Harrow',
    'Havering', 'Hillingdon', 'Hounslow', 'Islington',
    'Kensington & Chelsea', 'Kingston Upon Thames', 'Lambeth',
    'Lewisham', 'Merton', 'Newham', 'Redbridge',
    'Richmond Upon Thames', 'Southwark', 'Sutton',
    'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster',
    'City Of London'
]


df_london = df[df['Borough'].isin(london_boroughs)].copy()



borough_counts = df_london['Borough'].value_counts().reset_index()
borough_counts.columns = ['Borough', 'number_of_incidents']


severe_mask = df_london['Injury outcome'].astype(str).str.contains('taken to hospital', case=False, na=False)
severe_counts = df_london[severe_mask]['Borough'].value_counts().reset_index()
severe_counts.columns = ['Borough', 'number_of_severe_incidents']


result = pd.merge(borough_counts, severe_counts, on='Borough', how='left')
result['number_of_severe_incidents'] = result['number_of_severe_incidents'].fillna(0).astype(int)
result = result.sort_values('number_of_incidents', ascending=False).reset_index(drop=True)
result.to_csv('CSVForQuestion1.csv', index=False)





In [ ]:
import geopandas as gpd
import pandas as pd

gdf = gpd.read_file("LondonGeo.json", driver='TopoJSON', layer='boroughs')

gdf['longitude'] = gdf.centroid.x
gdf['latitude'] = gdf.centroid.y

centroids = gdf[['id', 'longitude', 'latitude']].copy()
centroids.columns = ['Borough', 'longitude', 'latitude']

incidents = pd.read_csv("CSVForQuestion1.csv")
merged = incidents.merge(centroids, on='Borough', how='left')


merged.to_csv("Centroids.csv", index=False) 




First line: Borough,number_of_incidents,number_of_severe_incidents,longitude,latitude



c:\Users\mason\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: driver TopoJSON does not support open option DRIVER
  return ogr_read(


In [12]:
import pandas as pd




df = pd.read_csv('injury-details-2025-q3.csv')

df = df[(df["Route"] != "Not specified") ]
def count_eq(series, value):
    return (series == value).sum()


grouped = df.groupby(["Route"]).apply(
    lambda g: pd.Series({
        "Total Incidents": len(g),
        "Slip Trip Fall": count_eq(g["Incident type"], "Slip Trip Fall"),
        "Collision": count_eq(g["Incident type"], "Collision"),
        "Assault": count_eq(g["Incident type"], "Assault"),
        "Male": count_eq(g["Gender"], "Male"),
        "Female": count_eq(g["Gender"], "Female"),
        "Youth (13 - 17)": count_eq(g["Age"], "Youth (13-17)"),
        "Adult (18-65)": count_eq(g["Age"], "Adult (18-65)"),
        "Child (0-12)" : count_eq(g["Age"], "Child (0-12)"),
        "Elderly (65+)": count_eq(g["Age"], "Elderly (65+)"),
        "Unknown Age": count_eq(g["Age"], "Unknown"),
        "Bus Driver": count_eq(g["Victim category"], "Bus Driver"),
        "Cyclist (Cycle Hire User)": count_eq(g["Victim category"], "Cyclist (Cycle Hire User)"),
        "Cyclist (Private Cycle)": count_eq(g["Victim category"], "Cyclist (Private Cycle)"),
        "Passenger / customer": count_eq(g["Victim category"], "Passenger / customer"),
        "Pedestrian": count_eq(g["Victim category"], "Pedestrian"),
    })
).reset_index()


top_15 = grouped.sort_values("Total Incidents", ascending=False).head(15)

top_15.to_csv("test.csv", index = False)


C:\Users\mason\AppData\Local\Temp\ipykernel_18640\2199444506.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby(["Route"]).apply(


In [20]:


df = pd.read_csv("injury-details-2025-q3.csv")


df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df['Day'] = df['Date'].dt.day_name()    
df['Month'] = df['Date'].dt.month_name()  


df['Day'] = df['Day'].fillna('Unknown')
df['Month'] = df['Month'].fillna('Unknown')


df = df.drop(columns=['Date'])

#df.head()


In [24]:



day_month_counts = (
    df.groupby(['Month', 'Day'])
      .size()
      .reset_index(name='Incidents')
)


day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
month_order = ['January','February','March','April','May','June',
               'July','August','September','October','November','December']

day_month_counts['Day'] = pd.Categorical(day_month_counts['Day'], categories=day_order, ordered=True)
day_month_counts['Month'] = pd.Categorical(day_month_counts['Month'], categories=month_order, ordered=True)

day_month_counts = day_month_counts.sort_values(['Month', 'Day'])

day_month_counts.to_csv("test2.csv", index=False)
